# Warm Distortion

Distortion can result in harsh signals. In order to combat this, there's several strategies to apply warmth and smoothing to the distorted signals

In [ ]:
import numpy as np
import warnings
from IPython.display import Audio, display
from scipy.io import wavfile
import matplotlib.pyplot as plt
from scipy import signal

def plot_waveform(waveform, sr, other=None, channel=0, start=0, end=2000, show_fft=False):
    """
    Plot a waveform (and optionally a second one for comparison).
    
    Args:
        waveform (np.ndarray): Array of shape (channels, samples)
        sr (int): Sample rate
        other (np.ndarray, optional): Second waveform to compare, same shape as waveform
        channel (int): Which channel to plot (default=0)
        start (int): Start sample index
        end (int): End sample index
        show_fft (bool): Whether to also plot frequency spectra
    """
    # Time axis in seconds
    samples = waveform.shape[1]
    end = min(end, samples)
    t = np.arange(start, end) / sr

    plt.figure(figsize=(12, 5 if not show_fft else 10))

    # --- Time-domain plot ---
    plt.subplot(2 if show_fft else 1, 1, 1)
    plt.plot(t, waveform[channel, start:end], label="Original")
    if other is not None:
        plt.plot(t, other[channel, start:end], label="Processed", alpha=0.8)
    plt.title(f"Waveform (Channel {channel})")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.legend()

    # --- Frequency-domain plot ---
    if show_fft:
        fft_orig = np.fft.rfft(waveform[channel])
        freqs = np.fft.rfftfreq(samples, 1/sr)
        plt.subplot(2, 1, 2)
        plt.semilogy(freqs, np.abs(fft_orig), label="Original")
        if other is not None:
            fft_proc = np.fft.rfft(other[channel])
            plt.semilogy(freqs, np.abs(fft_proc), label="Processed", alpha=0.5)
        plt.title("Frequency Spectrum")
        plt.xlabel("Frequency [Hz]")
        plt.ylabel("Magnitude")
        plt.legend()

    plt.tight_layout()
    plt.show()

# Original Source

In [ ]:
og_len = 5000 # 5 seconds
channels = 2  # Stereo audio
sr = 44100 # stream rate

distort_amount = 3  # Distortion amount
audio_path = "../../resources/synth1.wav"

warnings.simplefilter("ignore", wavfile.WavFileWarning)
sr_loaded, y = wavfile.read(audio_path)

# Convert to float32 and shape to (channels, samples)
waveform = y.T.astype(np.float32) / np.max(np.abs(y))  # normalize
num_samples = waveform.shape[1]

print(f"Original length: {num_samples} samples, Sample rate: {sr_loaded} Hz")
display(Audio(waveform, rate=sr))

# Distortion with output LPF

In [ ]:
# Apply distortion via tanh function
waveform_distorted = np.tanh(distort_amount * waveform)

# Low-pass filter parameters
cutoff_freq = 1500
order = 3
filter_type = 'lowpass'
filter_design_type = 'butter'

# Low-pass filter design (obtain coefficients)
sos = signal.iirfilter(order, cutoff_freq, btype=filter_type, ftype=filter_design_type, fs=sr, output='sos')
filtered_signal = signal.sosfilt(sos, waveform_distorted)

# Normalize to -1 < 0 < -1 to prevent clipping
max_val = np.max(np.abs(filtered_signal))
if max_val > 1.0:
    filtered_signal = filtered_signal / max_val

display(Audio(filtered_signal, rate=sr))
plot_waveform(waveform, sr, other=filtered_signal, channel=0, show_fft=True)



# Distortion with pre/post filters

In [ ]:
def high_shelf(cutoff_hz, sr, gain_db, Q=0.707):
    A = 10**(gain_db / 40)
    w0 = 2 * np.pi * cutoff_hz / sr
    alpha = np.sin(w0) / (2 * Q)
    cosw0 = np.cos(w0)

    b0 =    A*((A+1) + (A-1)*cosw0 + 2*np.sqrt(A)*alpha)
    b1 = -2*A*((A-1) + (A+1)*cosw0)
    b2 =    A*((A+1) + (A-1)*cosw0 - 2*np.sqrt(A)*alpha)
    a0 =       (A+1) - (A-1)*cosw0 + 2*np.sqrt(A)*alpha
    a1 =  2*((A-1) - (A+1)*cosw0)
    a2 =       (A+1) - (A-1)*cosw0 - 2*np.sqrt(A)*alpha

    # Normalize coefficients
    b = np.array([b0/a0, b1/a0, b2/a0])
    a = np.array([1.0, a1/a0, a2/a0])

    # Convert to sos for numerical stability
    sos = np.zeros((1, 6))
    sos[0, :3] = b
    sos[0, 3:] = a
    return sos

# Design a +24 dB high-shelf starting at 8khz
sos_input = high_shelf(cutoff_hz=8000, sr=sr, gain_db=24)
high_boost_signal = signal.sosfilt(sos, waveform)

# Apply distortion via tanh function
post_high_boost = np.tanh(distort_amount * high_boost_signal)

# Design a -24 dB high shelf cut at 8khz
sos_output = high_shelf(cutoff_hz=8000, sr=sr, gain_db=-24)
post_high_cut = signal.sosfilt(sos_output, post_high_boost)

# Normalize to -1 < 0 < -1 to prevent clipping
max_val = np.max(np.abs(post_high_cut))
if max_val > 1.0:
    waveform_lshelf = post_high_cut / max_val

display(Audio(post_high_cut, rate=sr))
plot_waveform(waveform, sr, other=post_high_cut, channel=0, show_fft=True)

import matplotlib.pyplot as plt
w, h = signal.sosfreqz(high_shelf(8000, sr, gain_db=24), worN=2000, fs=sr)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title("High shelf +24 dB @ 8 kHz")
plt.grid(True)
plt.show()

import matplotlib.pyplot as plt
w, h = signal.sosfreqz(high_shelf(8000, sr, gain_db=-24), worN=2000, fs=sr)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title("High shelf -24 dB @ 8 kHz")
plt.grid(True)
plt.show()

# Hard Distortion with pre/post filters

In [ ]:
distort_amount = 3

# Design a +24 dB high-shelf starting at 8khz
sos_input = high_shelf(cutoff_hz=8000, sr=sr, gain_db=24)
high_boost_signal = signal.sosfilt(sos, waveform)

# Apply distortion via hard clipping
post_high_boost = np.clip(distort_amount * high_boost_signal, -1, 1)
#post_high_boost = np.sign(high_boost_signal) * (1 - np.exp(-distort_amount * np.abs(high_boost_signal)))
#post_high_boost = np.tanh(distort_amount * high_boost_signal)

# Design a -24 dB high shelf cut at 8khz
sos_output = high_shelf(cutoff_hz=8000, sr=sr, gain_db=-24)
post_high_cut = signal.sosfilt(sos_output, post_high_boost)

# Normalize to -1 < 0 < -1 to prevent clipping
max_val = np.max(np.abs(post_high_cut))
if max_val > 1.0:
    waveform_lshelf = post_high_cut / max_val

display(Audio(post_high_cut, rate=sr))
plot_waveform(waveform, sr, other=post_high_cut, channel=0, show_fft=True)
